In [12]:
def get_library_path()->str:

    cwd = os.getcwd()
    lst = cwd.split(os.sep)[:-1]

    path_lib = f'{os.sep}'

    for elem in lst:
        path_lib = os.path.join(path_lib, elem)

    return path_lib

# imports

import os

import pandas as pd

path_to_lib = get_library_path()

In [13]:
# load data

df_3 = pd.read_csv(
    os.path.join(path_to_lib, 'data/auxiliar/aux_table_3.csv')
)
df_3 = df_3.drop(columns='Statistical Measure', inplace=False).fillna('')

df_51 = pd.read_csv(
    os.path.join(path_to_lib, 'data/auxiliar/aux_table_5_1.csv')
)
df_51 = df_51.drop(columns='Statistical Measure', inplace=False).fillna('')

df_71 = pd.read_csv(
    os.path.join(path_to_lib, 'data/auxiliar/aux_table_7_1.csv')
)
df_71 = df_71.drop(columns='Statistical Measure', inplace=False).fillna('')

In [14]:
df_71

,Variable,Tremor Dominant,Indeterminate,Postural instability and gait difficulty,Total,p-value,Adjusted p-value,Available Samples for Analysis
0,H & Y OFF,,,,,,,
1,Stage I-III,588 (98.3),338 (96.3),769 (90.7),1695 (94.3),p<0.001,p<0.001,1797.0
2,Stage IV-V,10 (1.7),13 (3.7),79 (9.3),102 (5.7),p<0.001,p<0.001,1797.0
3,H & Y ON,,,,,,,
4,Stage I-III,,,,,,,
5,Stage VI-V,,,,,,,
6,UPDRS III-ON,2.0 (2.0 - 2.0),9.0 (9.0 - 9.0),46.0 (46.0 - 46.0),9.0 (5.5 - 27.5),0.3679,0.0085,3.0
7,UPDRS III-OFF,35.0 (25.0 - 46.0),39.0 (31.0 - 46.0),39.0 (28.0 - 48.0),38.0 (27.0 - 47.0),0.0013,0.9999,1473.0
8,MOCA Score,25.3 (6.1),24.8 (6.0),24.2 (5.8),24.7 (6.0),0.0054,0.0039,1533.0
9,BDI Score,8.0 (4.0 - 12.0),9.0 (5.0 - 13.0),9.0 (6.0 - 14.0),9.0 (5.0 - 13.0),p<0.001,p<0.001,1646.0


In [15]:
# create multi-level columns for Table 3

multi_index_3 = pd.MultiIndex.from_tuples(
    [('', 'Variables'), 
     ('Male/Female Comp', 'Male'), 
     ('Male/Female Comp', 'Female'),
     ('Male/Female Comp', 'p-value'),
     ('Male/Female Comp', 'Total'),
     ('Male/Female Comp', 'Available Samples for Analysis')]
)
df_3.columns = multi_index_3

In [16]:
# create multi-level columns for Table 6

multi_index_51 = pd.MultiIndex.from_tuples(
    [('Onset', 'Variables'), 
     ('Early/Late Onset Comp', 'Age at onset <50 years'), 
     ('Early/Late Onset Comp', 'Age at onset >=50 years'),
     ('Early/Late Onset Comp', 'p-value'),
     ('Early/Late Onset Comp', 'Total'),
     ('Early/Late Onset Comp', 'Available Samples for Analysis')]
)
df_51.columns = multi_index_51

In [17]:
# create multi-level columns for Table 8

multi_index_71 = pd.MultiIndex.from_tuples(
    [('Subtype', 'Variables'), 
     ('Motor Subtype', 'PIGD'), 
     ('Motor Subtype', 'TD'),
     ('Motor Subtype', 'ID'),
     ('Motor Subtype', 'Total'),
     ('Motor Subtype', 'p-value'),
     ('Motor Subtype', 'Adjusted p-value'),
     ('Motor Subtype', 'Available Samples for Analysis')]
)
df_71.columns = multi_index_71

In [18]:
# merge and save resulting table

df_mrg = pd.concat([df_3, df_51, df_71], axis=1)\
    .drop(columns=[('Onset', 'Variables'), ('Subtype', 'Variables')], inplace=False)

df_mrg

Male/Female Comp                               \
       Variables                Male              Female  p-value   
0      H & Y OFF                                                    
1    Stage I-III         1642 (94.4)          795 (93.4)   0.3377   
2     Stage IV-V            98 (5.6)            56 (6.6)   0.3377   
3       H & Y ON                                                    
4    Stage I-III         2537 (95.8)         1200 (95.4)    0.584   
5     Stage IV-V           112 (4.2)            58 (4.6)    0.584   
6   UPDRS III-ON  21.0 (13.0 - 30.0)  20.0 (13.0 - 29.0)   0.0582   
7  UPDRS III-OFF  38.0 (27.0 - 46.0)  37.0 (26.0 - 46.0)   0.3854   
8     MOCA Score          24.1 (5.2)          22.8 (6.2)  p<0.001   
9      BDI Score    8.0 (5.0 - 13.0)    9.0 (6.0 - 14.0)  p<0.001   

                                                      Early/Late Onset Comp  \
                Total Available Samples for Analysis Age at onset <50 years   
0                                                                             
1         2437 (94.1)                         2591.0             783 (92.6)   
2           154 (5.9)                         2591.0               63 (7.4)   
3                                                                             
4         3737 (95.6)                         3907.0            1260 (96.1)   
5           170 (4.4)                         3907.0               51 (3.9)   
6  21.0 (13.0 - 30.0)                         3223.0     20.0 (12.0 - 28.0)   
7  37.0 (27.0 - 46.0)                         1562.0     38.0 (27.0 - 47.0)   
8          23.7 (5.5)                         5301.0             24.2 (5.5)   
9    9.0 (5.0 - 13.0)                         5850.0       9.0 (6.0 - 14.0)   

                                                        \
  Age at onset >=50 years  p-value               Total   
0                                                        
1             1564 (94.7)   0.0302         2347 (94.0)   
2                87 (5.3)   0.0302           150 (6.0)   
3                                                        
4             2400 (95.5)   0.3518         3660 (95.7)   
5               114 (4.5)   0.3518           165 (4.3)   
6      21.0 (13.0 - 30.0)   0.0038  21.0 (13.0 - 30.0)   
7      37.0 (27.0 - 46.0)   0.6416  37.0 (27.0 - 46.0)   
8              23.5 (5.5)  p<0.001          23.7 (5.5)   
9        8.0 (5.0 - 13.0)  p<0.001    9.0 (5.0 - 13.0)   

                                       Motor Subtype                      \
  Available Samples for Analysis                PIGD                  TD   
0                                                                          
1                         2497.0          588 (98.3)          338 (96.3)   
2                         2497.0            10 (1.7)            13 (3.7)   
3                                                                          
4                         3825.0                                           
5                         3825.0                                           
6                         3166.0     2.0 (2.0 - 2.0)     9.0 (9.0 - 9.0)   
7                         1546.0  35.0 (25.0 - 46.0)  39.0 (31.0 - 46.0)   
8                         5147.0          25.3 (6.1)          24.8 (6.0)   
9                         5698.0    8.0 (4.0 - 12.0)    9.0 (5.0 - 13.0)   

                                                                     \
                   ID               Total  p-value Adjusted p-value   
0                                                                     
1          769 (90.7)         1695 (94.3)  p<0.001          p<0.001   
2            79 (9.3)           102 (5.7)  p<0.001          p<0.001   
3                                                                     
4                                                                     
5                                                                     
6  46.0 (46.0 - 46.0)    9.0 (5.5 - 27.5)   0.3679           0.0

In [20]:
df_mrg.to_csv(os.path.join(path_to_lib, 'data/final/Table_3.csv'), index=False)